In [ ]:
import findspark
findspark.init()

from pyspark import SparkContext
#Objetos que dan formato al dataframe
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import IntegerType, StringType, FloatType
from pyspark.sql.types import Row
from pyspark.sql import SQLContext

#Mala práctica hacer Import *
import pyspark.sql.functions

In [ ]:
sc = SparkContext(master="local", appName="pruebasCP")

In [ ]:
#Se llama
sqlContext = SQLContext(sc)

In [ ]:
#Se genera una variable que contiene la ruta donde se encuentran los
var = "C:/Users/Caroline/Videos/curso-apache-spark-platzi/files/"

deptaRDD = sc.textFile(var+"deportista.csv").map(lambda line : line.split(","))
equiRDD  = sc.textFile(var+"paises.csv").map(lambda line : line.split(","))

In [ ]:
#Función para validar el encabezado
#Iter: Devuelve valor a valor lo que se está procesando
def deletHeader(indice, interador):
    return iter(list(interador)[1:])

In [ ]:
#Eliminar el encabezado
deptaRDD = deptaRDD.mapPartitionsWithIndex(deletHeader) #Deportista
equiRDD = equiRDD.mapPartitionsWithIndex(deletHeader) #Equipo

In [ ]:
#Antes de Tramsformar los datos del RDD, se debe transformar el tipo de dato
#Deportista
deptaRDD = deptaRDD.map(lambda l: (
l[0],
l[1],    
l[2],
l[3],
l[4],
l[5],
l[6]
))


#Equipos
equiRDD = equiRDD.map(lambda l: (
int(l[0]),
l[1],
l[2]
))

In [ ]:
#Se crear el Schema
#Deportista
depta = StructType([
    StructField("deportista_id", IntegerType(), False),
    StructField("nombre_deportista", StringType(), False),
    StructField("genero", IntegerType(), False),
    StructField("edad", IntegerType(), False),
    StructField("altura", FloatType(), False),
    StructField("peso", FloatType(), False),
    StructField("equipo_id", IntegerType(), False)
])


#Equipos
equi = StructType([
    StructField("equipo_id", IntegerType(), False),
    StructField("nombre_equipo", StringType(), False),
    StructField("sigla", StringType(), False)
])

In [ ]:
#Aplicar el esquema al RDD
#Deportista
deptaRDD = sqlContext.createDataFrame(deptaRDD, depta)

#Equipo
equiRDD = sqlContext.createDataFrame(equiRDD, equi)

In [ ]:
#Renombrar columnas y eliminar columnas
#datRDD = datRDD.withColumnRenamed("annio", "anho")

In [ ]:
#Mostrar esquema
#datRDD.printSchema()

In [ ]:
#Seleccionar sólo algunas columnas
#Función col--> Genera una lista en vivo, contiene todas las columnas, 
#para realizar operaciones complejas
#datRDD.select("id", "nombre_juego", "ciudad")

In [ ]:
#Filtros
#datRDD = datRDD.filter((datRDD.anho > 1900))

In [ ]:
#Mostrar datos
#Deportista
deptaRDD.show()

In [ ]:
#Mostrar datos
#Equipo
#Filtrar datos a visualizar
#equiRDD.filter((equiRDD.equipo_id == 273)).show()
equiRDD.show()

In [ ]:
#Join entre tablas
deptaRDD.join(
    equiRDD, #Nombre de tabla del JOIN
    deptaRDD.equipo_id == equiRDD.equipo_id, #Campos de relación
    "left" #Tipo de JOIN
).select(deptaRDD.nombre_deportista, deptaRDD.edad, equiRDD.sigla, equiRDD.nombre_equipo).filter((equiRDD.equipo_id == 273)).show()